# Importações de dados brutos

In [214]:
import os
import joblib

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from scipy import stats
from scipy.stats import mannwhitneyu, normaltest, ranksums, chisquare
from statsmodels.stats.weightstats import zconfint, DescrStatsW, ztest

In [215]:
!pip install util-gfsilveira

In [216]:
from util import *

In [217]:
meus_uteis()

-------------
-=< Lista >=-
-------------
0 -> printLinha()
1 -> printLis()
2 -> printDic()
3 -> timeProcess()
4 -> import_started()
5 -> img_save()
6 -> setup_img_save()
7 -> mask_corr_graphic()
8 -> structured_confusion()


In [218]:
os.chdir('dados')

In [219]:
perguntas = joblib.load('perguntas.gz')
df = joblib.load('df_mysql.gz')
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [220]:
# df.to_csv('dados_completos.csv')

In [221]:
perguntas

{'Compreensão': '6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.',
 'Descrição': '8) Existem elementos da imagem que requerem descrição adicional?',
 'Dintinguíveis': '1) Na imagem, o quão claramente os elementos são distinguíveis?',
 'Eficiente': '3) O quão eficiente são os elementos da imagem na representação dos dados?',
 'Estética': '5) O quanto a imagem é esteticamente bem elaborada?',
 'Fonte': '9) A imagem apresentada faz você experimentar uma nova fonte de dados?',
 'Grau': 'Qual o seu grau de formação acadêmica?',
 'Interesse': '7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.',
 'Proeminentes': '2) O quão visualmente proeminentes são os elementos da imagem?',
 'Área': 'Área de formação?',
 'Útil': '4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?'}

# Geral

In [222]:
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [223]:
df.describe()

,Dintinguíveis,Proeminentes,Eficiente,Útil,Estética,Compreensão,Interesse
count,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000
mean,3.786017,3.576271,3.552966,4.614407,3.567797,3.599576,3.489407
std,1.227245,1.274654,1.247227,0.909751,1.340082,1.314075,1.376129
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,5.000000,3.000000,3.000000,2.000000
50%,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 472
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Data           472 non-null    datetime64[ns]
 1   IP             472 non-null    object        
 2   TCLE           472 non-null    object        
 3   Língua         472 non-null    object        
 4   Figura         472 non-null    object        
 5   Dintinguíveis  472 non-null    int32         
 6   Proeminentes   472 non-null    int32         
 7   Descrição      472 non-null    object        
 8   Eficiente      472 non-null    int32         
 9   Útil           472 non-null    int32         
 10  Estética       472 non-null    int32         
 11  Fonte          472 non-null    object        
 12  Compreensão    472 non-null    int32         
 13  Interesse      472 non-null    int32         
 14  Grau           472 non-null    object        
 15  Área           472 non-

# Agrupando

## Analisando a pergunta Descrição

### Geral

In [225]:
colunas_perguntas = df.columns[[7,11,14,15,16]]
df_quantitativo = df[colunas_perguntas]
df_quantitativo.head()

,Descrição,Fonte,Grau,Área,Origem
id,,,,,
1,Sim,Sim,Doutorado,Exatas,Pesquisador
2,Sim,Não,Doutorado,Biológicas,Designer
3,Sim,Não,Doutorado,Exatas,Pesquisador
4,Sim,Sim,Mestrado,Biológicas,Designer
5,Sim,Sim,Doutorado,Biológicas,Pesquisador


In [226]:
df_descricao = df_quantitativo[colunas_perguntas[[0,4]]].value_counts(normalize=False)
df_descricao

Descrição  Origem     
Sim        Pesquisador    206
           Designer       168
Não        Designer        68
           Pesquisador     30
dtype: int64

In [227]:
dic_descricao = {
    'Designer': [
                    df_descricao.loc['Sim',:].loc['Designer'],
                    df_descricao.loc['Não',:].loc['Designer'],
                 ],
    'Pesquisador': [
                      df_descricao.loc['Sim',:].loc['Pesquisador'],
                      df_descricao.loc['Não',:].loc['Pesquisador'],
                    ],
}
dic_descricao

{'Designer': [168, 68], 'Pesquisador': [206, 30]}

In [228]:
index=['Sim','Não']
df_descricao_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_descricao.keys())
df_descricao_compila['Pesquisador'] = dic_descricao['Pesquisador']
df_descricao_compila['Designer'] = dic_descricao['Designer']
df_descricao_compila

,Designer,Pesquisador
Sim,168,206
Não,68,30


In [229]:
frequencias = [[df_descricao_compila.loc['Sim','Pesquisador'],df_descricao_compila.loc['Não','Pesquisador']],
              [df_descricao_compila.loc['Sim','Designer'],df_descricao_compila.loc['Não','Designer']]]
frequencias

[[206, 30], [168, 68]]

In [230]:
_, p_valor = chisquare(f_exp=df_descricao_compila['Pesquisador'], f_obs=df_descricao_compila['Designer'])
if p_valor < .001:
  p_valor = '<0.001'

df_descricao_compila['p-valor'] = (p_valor, np.nan)
# df_descricao_compila.to_csv('df_descricao_compila.csv')
df_descricao_compila

,Designer,Pesquisador,p-valor
Sim,168,206,<0.001
Não,68,30,NaN


### Área

#### Biológicas

In [231]:
def tabela_freq_qualitattivo(df_teste):
  df_fonte = df_teste.value_counts(normalize=False)
  dic_fonte = {
      'Designer': [
                      df_fonte.loc['Sim',:].loc['Designer'],
                      df_fonte.loc['Não',:].loc['Designer'],
                  ],
      'Pesquisador': [
                        df_fonte.loc['Sim',:].loc['Pesquisador'],
                        df_fonte.loc['Não',:].loc['Pesquisador'],
                      ],
  }
  index=['Sim','Não']
  df_fonte_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_fonte.keys())
  df_fonte_compila['Pesquisador'] = dic_fonte['Pesquisador']
  df_fonte_compila['Designer'] = dic_fonte['Designer']
  df_fonte_compila = df_fonte_compila[df_fonte_compila.columns[[1,0]]]
  frequencias = [[df_fonte_compila.loc['Sim','Pesquisador'],df_fonte_compila.loc['Não','Pesquisador']],
                [df_fonte_compila.loc['Sim','Designer'],df_fonte_compila.loc['Não','Designer']]]
  _, p_valor = chisquare(f_exp=df_fonte_compila['Pesquisador'], f_obs=df_fonte_compila['Designer'])
  if p_valor < .001:
    p_valor = '<0.001'

  df_fonte_compila['p-valor'] = (p_valor, np.nan)
  return df_fonte_compila

In [232]:
df_descricao_compila_bio = df_quantitativo[df_quantitativo["Área"] == "Biológicas"][colunas_perguntas[[0,4]]]
df_descricao_compila_bio.value_counts()

Descrição  Origem     
Sim        Pesquisador    161
           Designer       130
Não        Designer        48
           Pesquisador     18
dtype: int64

In [233]:
df_descricao_compila_bio = tabela_freq_qualitattivo(df_descricao_compila_bio)
# df_descricao_compila_bio.to_csv("df_descricao_compila_bio.csv")
df_descricao_compila_bio

,Pesquisador,Designer,p-valor
Sim,161,130,<0.001
Não,18,48,NaN


#### Exatas

In [234]:
df_descricao_compila_exatas = df_quantitativo[df_quantitativo["Área"] == "Exatas"][colunas_perguntas[[0,4]]]
df_descricao_compila_exatas.value_counts()

Descrição  Origem     
Sim        Designer       23
           Pesquisador    20
Não        Designer       11
           Pesquisador     3
dtype: int64

In [235]:
df_descricao_compila_exatas = tabela_freq_qualitattivo(df_descricao_compila_exatas)
# df_descricao_compila_exatas.to_csv("df_descricao_compila_exatas.csv")
df_descricao_compila_exatas

,Pesquisador,Designer,p-valor
Sim,20,23,<0.001
Não,3,11,NaN


#### Humanas

In [236]:
df_descricao_compila_humanas = df_quantitativo[df_quantitativo["Área"] == "Humanas"][colunas_perguntas[[0,4]]]
df_descricao_compila_humanas.value_counts()

Descrição  Origem     
Sim        Pesquisador    25
           Designer       15
Não        Designer        9
           Pesquisador     9
dtype: int64

In [237]:
df_descricao_compila_humanas = tabela_freq_qualitattivo(df_descricao_compila_humanas)
# df_descricao_compila_humanas.to_csv("df_descricao_compila_humanas.csv")
df_descricao_compila_humanas

,Pesquisador,Designer,p-valor
Sim,25,15,0.0455
Não,9,9,NaN


### Grau

#### Graduação

In [238]:
df_descricao_compila_gradu = df_quantitativo[df_quantitativo["Grau"] == "Graduação"][colunas_perguntas[[0,4]]]
df_descricao_compila_gradu.value_counts()

Descrição  Origem     
Sim        Designer       18
           Pesquisador    17
Não        Designer        8
           Pesquisador     2
dtype: int64

In [239]:
df_descricao_compila_gradu = tabela_freq_qualitattivo(df_descricao_compila_gradu)
# df_descricao_compila_gradu.to_csv("df_descricao_compila_gradu.csv")
df_descricao_compila_gradu

,Pesquisador,Designer,p-valor
Sim,17,18,<0.001
Não,2,8,NaN


#### Pós-Graduação

In [240]:
df_descricao_compila_posGradu = df_quantitativo[df_quantitativo["Grau"] == "Pós-Graduação"][colunas_perguntas[[0,4]]]
df_descricao_compila_posGradu.value_counts()

Descrição  Origem     
Sim        Designer       33
           Pesquisador    31
Não        Designer       14
           Pesquisador     7
dtype: int64

In [241]:
df_descricao_compila_posGradu = tabela_freq_qualitattivo(df_descricao_compila_posGradu)
# df_descricao_compila_posGradu.to_csv("df_descricao_compila_posGradu.csv")
df_descricao_compila_posGradu

,Pesquisador,Designer,p-valor
Sim,31,33,0.007585
Não,7,14,NaN


#### Mestrado

In [242]:
df_descricao_compila_mestrado = df_quantitativo[df_quantitativo["Grau"] == "Mestrado"][colunas_perguntas[[0,4]]]
df_descricao_compila_mestrado.value_counts()

Descrição  Origem     
Sim        Pesquisador    89
           Designer       62
Não        Designer       23
           Pesquisador    13
dtype: int64

In [243]:
df_descricao_compila_mestrado = tabela_freq_qualitattivo(df_descricao_compila_mestrado)
# df_descricao_compila_mestrado.to_csv("df_descricao_compila_mestrado.csv")
df_descricao_compila_mestrado

,Pesquisador,Designer,p-valor
Sim,89,62,<0.001
Não,13,23,NaN


#### Doutorado

In [244]:
df_descricao_compila_doc = df_quantitativo[df_quantitativo["Grau"] == "Doutorado"][colunas_perguntas[[0,4]]]
df_descricao_compila_doc.value_counts()

Descrição  Origem     
Sim        Pesquisador    69
           Designer       55
Não        Designer       23
           Pesquisador     8
dtype: int64

In [245]:
df_descricao_compila_doc = tabela_freq_qualitattivo(df_descricao_compila_doc)
# df_descricao_compila_doc.to_csv("df_descricao_compila_doc.csv")
df_descricao_compila_doc

,Pesquisador,Designer,p-valor
Sim,69,55,<0.001
Não,8,23,NaN


## Analisando a pergunta Fonte

### Geral

In [246]:
df_fonte_compila = df_quantitativo[colunas_perguntas[[1,4]]]
df_fonte_compila.value_counts()

Fonte  Origem     
Sim    Designer       191
       Pesquisador    146
Não    Pesquisador     90
       Designer        45
dtype: int64

In [247]:
df_fonte_compila = tabela_freq_qualitattivo(df_fonte_compila)
# df_fonte_compila.to_csv("df_fonte_compila.csv")
df_fonte_compila

,Pesquisador,Designer,p-valor
Sim,146,191,<0.001
Não,90,45,NaN


### Área

#### Bilógicas

In [248]:
df_fonte_compila_bio = df_quantitativo[df_quantitativo["Área"] == "Biológicas"][colunas_perguntas[[1,4]]]
df_fonte_compila_bio.value_counts()

Fonte  Origem     
Sim    Designer       140
       Pesquisador    104
Não    Pesquisador     75
       Designer        38
dtype: int64

In [249]:
df_fonte_compila_bio = tabela_freq_qualitattivo(df_fonte_compila_bio)
# df_fonte_compila_bio.to_csv("df_fonte_compila_bio.csv")
df_fonte_compila_bio

,Pesquisador,Designer,p-valor
Sim,104,140,<0.001
Não,75,38,NaN


#### Exatas

In [250]:
df_fonte_compila_exatas = df_quantitativo[df_quantitativo["Área"] == "Exatas"][colunas_perguntas[[1,4]]]
df_fonte_compila_exatas.value_counts()

Fonte  Origem     
Sim    Designer       30
       Pesquisador    14
Não    Pesquisador     9
       Designer        4
dtype: int64

In [251]:
df_fonte_compila_exatas = tabela_freq_qualitattivo(df_fonte_compila_exatas)
# df_fonte_compila_exatas.to_csv("df_fonte_compila_exatas.csv")
df_fonte_compila_exatas

,Pesquisador,Designer,p-valor
Sim,14,30,<0.001
Não,9,4,NaN


#### Humanas

In [262]:
df_fonte_compila_humanas = df_quantitativo[df_quantitativo["Área"] == "Humanas"][colunas_perguntas[[1,4]]]
df_fonte_compila_humanas.value_counts()

Fonte  Origem     
Sim    Pesquisador    28
       Designer       21
Não    Pesquisador     6
       Designer        3
dtype: int64

In [263]:
df_fonte_compila_humanas = tabela_freq_qualitattivo(df_fonte_compila_humanas)
# df_fonte_compila_humanas.to_csv("df_fonte_compila_humanas.csv")
df_fonte_compila_humanas

,Pesquisador,Designer,p-valor
Sim,28,21,0.071423
Não,6,3,NaN


### Grau

#### Graduação

In [254]:
df_fonte_compila_gradua = df_quantitativo[df_quantitativo["Grau"] == "Graduação"][colunas_perguntas[[1,4]]]
df_fonte_compila_gradua.value_counts()

Fonte  Origem     
Sim    Designer       22
       Pesquisador    10
Não    Pesquisador     9
       Designer        4
dtype: int64

In [255]:
df_fonte_compila_gradua = tabela_freq_qualitattivo(df_fonte_compila_gradua)
# df_fonte_compila_gradua.to_csv("df_fonte_compila_gradua.csv")
df_fonte_compila_gradua

,Pesquisador,Designer,p-valor
Sim,10,22,<0.001
Não,9,4,NaN


#### Pós-Graduação

In [256]:
df_fonte_compila_posGradua = df_quantitativo[df_quantitativo["Grau"] == "Pós-Graduação"][colunas_perguntas[[1,4]]]
df_fonte_compila_posGradua.value_counts()

Fonte  Origem     
Sim    Designer       37
       Pesquisador    27
Não    Pesquisador    11
       Designer       10
dtype: int64

In [257]:
df_fonte_compila_posGradua = tabela_freq_qualitattivo(df_fonte_compila_posGradua)
# df_fonte_compila_posGradua.to_csv("df_fonte_compila_posGradua.csv")
df_fonte_compila_posGradua

,Pesquisador,Designer,p-valor
Sim,27,37,0.051418
Não,11,10,NaN


#### Mestrado

In [258]:
df_fonte_compila_mestrado = df_quantitativo[df_quantitativo["Grau"] == "Mestrado"][colunas_perguntas[[1,4]]]
df_fonte_compila_mestrado.value_counts()

Fonte  Origem     
Sim    Designer       73
       Pesquisador    59
Não    Pesquisador    43
       Designer       12
dtype: int64

In [259]:
df_fonte_compila_mestrado = tabela_freq_qualitattivo(df_fonte_compila_mestrado)
# df_fonte_compila_mestrado.to_csv("df_fonte_compila_mestrado.csv")
df_fonte_compila_mestrado

,Pesquisador,Designer,p-valor
Sim,59,73,<0.001
Não,43,12,NaN


#### Doutorado

In [260]:
df_fonte_compila_doc = df_quantitativo[df_quantitativo["Grau"] == "Doutorado"][colunas_perguntas[[1,4]]]
df_fonte_compila_doc.value_counts()

Fonte  Origem     
Sim    Designer       59
       Pesquisador    50
Não    Pesquisador    27
       Designer       19
dtype: int64

In [261]:
df_fonte_compila_doc = tabela_freq_qualitattivo(df_fonte_compila_doc)
# df_fonte_compila_doc.to_csv("df_fonte_compila_doc.csv")
df_fonte_compila_doc

,Pesquisador,Designer,p-valor
Sim,50,59,0.045761
Não,27,19,NaN
